In [ ]:
!pip install fastapi uvicorn pyngrok transformers accelerate bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from pyngrok import ngrok

# Konfigurasi Folder Model
MODEL_DIR = "/content/drive/MyDrive/qwen2-7b-instruct-4bit"
os.makedirs(MODEL_DIR, exist_ok=True)

In [ ]:
%%writefile model_utils.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

def load_qwen_model(model_dir):
    print("Memuat model Qwen2-7B-Instruct dari:", model_dir)

    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        quantization_config=quant_config,
        device_map="auto",
        trust_remote_code=True
    )

    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return tokenizer, pipe

Writing model_utils.py


Writing requirements.txt


In [ ]:
from fastapi import FastAPI, Request
import re
from model_utils import load_qwen_model

tokenizer, pipe = load_qwen_model(MODEL_DIR)
# load_qwen_model("/content/drive/MyDrive/qwen2-7b-instruct-4bit")
# load_qwen_model(MODEL_DIR)

app = FastAPI()

Memuat model Qwen2-7B-Instruct dari: /content/drive/MyDrive/qwen2-7b-instruct-4bit


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
# Menentukan Aspek
KATEGORI_ASPEK = [
    "Kamera", "Baterai", "Performa & Gaming", "Layar",
    "Desain & Kualitas Bodi", "Harga", "Pengiriman",
    "Layanan Penjual", "UI"
]

# Template Prompt
def buat_prompt_few_shot(ulasan):
    return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat.
Tugas Anda adalah mengidentifikasi SEMUA aspek relevan dari ulasan pengguna berikut.
DAFTAR ASPEK RESMI: {KATEGORI_ASPEK}

PENTING:
- JANGAN pernah menyebut aspek di luar daftar di atas.
- Contoh aspek ILEGAL: "Kinerja", "Kualitas Barang", "Garansi" — ini BUKAN nama aspek resmi.
- Format jawaban HARUS berupa list Python TANPA penjelasan tambahan.
- Jawaban HARUS dimulai dengan `[` dan diakhiri `]`.

CONTOH:
Ulasan: "Kamera depan jelek, Kinerja mantap, baterai tahan seharian."
Aspek: ["Kamera", "Performa & Gaming", "Baterai"]

Ulasan: "Pengiriman cepat, Fungsi sesuai, Garansi ada."
Aspek: ["Pengiriman", "Performa & Gaming", "Layanan Penjual"]

Ulasan: "Kondisi Barang Baik, harga terlalu mahal, Penjual ramah."
Aspek: ["Desain & Kualitas Bodi", "Harga", "Layanan Penjual"]

Ulasan: "mantapp, dipake berjam jam ga mati mati, durasinya tahan lama ini"
Aspek: ["Baterai"]

Ulasan: "Terimakasih Banyak, mantap"
Aspek: [""]

Sekarang, kerjakan ulasan berikut.
Ulasan: "{ulasan}"
Aspek:"""

def buat_prompt_few_shot_cot(ulasan):
    return f"""Anda adalah seorang analis ulasan produk smartphone yang akurat. Tugas Anda adalah mengidentifikasi semua aspek relevan dari ulasan pengguna berikut. Anda HANYA BOLEH memilih aspek dari daftar ini: {KATEGORI_ASPEK}. Jika frasa tidak secara eksplisit menyebut nama aspek, pilih aspek yang paling relevan dari daftar.

CONTOH:

Ulasan: "Kondisi Barang: Baru Original
Garansi: Original
Kinerja: Berfungsi dengan baik
Mantab barang yang ditunggu2 datang, packaging aman, barang baru dan bersebelahan, berfungsi dengan normal"
Identifikasi Frasa Kunci:
- "Kondisi Barang: Baru Original", "barang baru" menyebutkan "Kondisi Barang", sehingga membahas → Desain & Kualitas Bodi
- "Garansi: Original" menyebutkan "Garansi", sehingga membahas → Layanan Penjual
- "Kinerja: Berfungsi dengan baik", "berfungsi dengan normal" menyebutkan "Kinerja" dan "berfungsi dengan normal", sehingga membahas → Performa & Gaming
- "packaging aman" menyebutkan "packaging", sehingga membahas → Pengiriman
Jawaban Akhir: ["Desain & Kualitas Bodi", "Layanan Penjual", "Performa & Gaming", "Pengiriman"]

Ulasan: "Barang sampai dengan selamat, packing rapih. HP-nya ngebut banget buat main ML, tapi agak cepet panas. Layar jernih, kamera depannya biasa aja, menarik banget,Suhu belum di test secara maksimal sih, POCO nya beraksi."
Identifikasi Frasa Kunci:
- "Barang sampai dengan selamat, packing rapih" menyebutkan "sampai dengan selamat" dan "packing", sehingga membahas → Pengiriman
- "HP-nya ngebut banget buat main ML" menyebutkan "ngebut banget" dan "main ML", sehingga membahas → Performa & Gaming
- "Layar jernih" menyebutkan "Layar", sehingga membahas → Layar
- "kamera depannya biasa aja" menyebutkan "kamera", sehingga membahas → Kamera
- "Suhu belum di test secara maksimal sih, POCO nya beraksi" TIDAK menyebutkan apa apa yang termasuk di kategori, sehingga tidak membahas aspek apapun
Jawaban Akhir: ["Pengiriman", "Performa & Gaming", "Layar", "Kamera"]

Ulasan: "Kualitas: Best value phone in the market right now
Garansi: panjang
Kondisi Barang: 100% aman mulus
hands down sama Poco F7! Performanya mantep abis, multitasking lancar jaya, game berat juga nggak ngelag Layar AMOLED nya keren banget, warna-warna cerah dan tajam, bikin gue makin asik nonton YouTube atau main game, Kamera 50MP nya juga jos, Foto-foto jernih, detail, dan nggak kalah sama kamera profesional! Baterai 650MAH nya awet banget, gue bisa main game seharian tanpa perlu khawatir kehabisan daya. Dan pengisian cepat 90Watt ? Gue bisa balik main dalam waktu singkat, Harganya juga nggak bikin kantong jebol, worth it banget"
Identifikasi Frasa Kunci:
- "Kualitas: Best value phone in the market right now", "Performanya mantep abis, multitasking lancar jaya, game berat juga nggak ngelag" menyebutkan "Kualitas", "Performanya", dan "game berat juga nggak ngelag", sehingga membahas → Performa & Gaming
- "Garansi: panjang" menyebutkan "Garansi", sehingga membahas → Layanan Penjual
- "Kondisi Barang: 100% aman mulus" menyebutkan "Kondisi Barang", sehingga membahas → Desain & Kualitas Bodi
- "Layar AMOLED nya keren banget, warna-warna cerah dan tajam" menyebutkan "Layar AMOLED", sehingga membahas → Layar
- "Kamera 50MP nya juga jos, Foto-foto jernih, detail, dan nggak kalah sama kamera profesional!" menyebutkan "Kamera", "Foto", sehingga membahas → Kamera
- "Baterai 650MAH nya awet banget, gue bisa main game seharian tanpa perlu khawatir kehabisan daya. Dan pengisian cepat 90Watt ?" menyebutkan "Baterai 650MAH nya awet", "tanpa perlu khawatir kehabisan daya", "pengisian cepat 90Watt", sehingga membahas → Baterai
- "Harganya juga nggak bikin kantong jebol" menyebutkan "Harganya", sehingga membahas → Harga

Jawaban Akhir: ["Performa & Gaming", "Layanan Penjual", "Desain & Kualitas Bodi", "Layar", "Kamera", "Baterai", "Harga"]

Sekarang, lakukan hal yang sama untuk ulasan berikut.
Ulasan: "{ulasan}"
Identifikasi Frasa Kunci:"""

# PEMBERSIHAN HASIL
def bersihkan_respons(teks):
    pola = r'\[(.*?)\]'
    hasil = re.search(pola, teks, re.DOTALL)
    if hasil:
        isi = hasil.group(1)
        aspek_list = [a.strip().strip('"').strip("'") for a in isi.split(",") if a.strip()]
        aspek_valid = [a for a in aspek_list if a in KATEGORI_ASPEK]
        return aspek_valid
    return []

def ekstrak_jawaban_akhir_dari_cot(teks):
    pola = r"Jawaban\s*Akhir(?: Contoh)?:?\s*(\[.*?\])"
    hasil = re.search(pola, teks, re.IGNORECASE | re.DOTALL)
    if hasil:
        return bersihkan_respons(hasil.group(1))
    return bersihkan_respons(teks)

# Rules Engine
def pilih_teknik(panjang_teks):
    return 'few_shot' if panjang_teks <= 7 else 'few_shot_cot'

# FUNGSI UTAMA ANALISIS
def analisis_ulasan(pipe, tokenizer, ulasan):
    teknik = pilih_teknik(len(ulasan.split()))
    prompt = buat_prompt_few_shot(ulasan) if teknik == 'few_shot' else buat_prompt_few_shot_cot(ulasan)

    messages = [{"role": "user", "content": prompt}]
    prompt_formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    outputs = pipe(
        prompt_formatted,
        max_new_tokens=16384,
        do_sample=False
        # do_sample=True,
        # temperature=1,
        # top_p=0.9
    )
    teks = outputs[0]["generated_text"][len(prompt_formatted):]

    hasil_bersih = (
        bersihkan_respons(teks)
        if teknik == "few_shot"
        else ekstrak_jawaban_akhir_dari_cot(teks)
    )

    hasil_unik = list(dict.fromkeys(hasil_bersih))

    print(f"\n Teknik Prompt: {teknik}")
    print(f"Output mentah:\n{teks}\n")
    print(f"Hasil akhir: {hasil_unik}\n")

    return {"prediksi": hasil_unik, "teknik": teknik}

@app.post("/analisis")
async def analisis_aspek(request: Request):
    data = await request.json()
    ulasan = data.get("ulasan", "")
    if not ulasan.strip():
        return {"error": "Teks ulasan kosong."}

    hasil = analisis_ulasan(pipe, tokenizer, ulasan)
    return hasil

@app.get("/")
def root():
    return {"status": "Server berjalan, siap menerima request."}


In [ ]:
from pyngrok import ngrok
import threading
import uvicorn

ngrok.kill()

ngrok.set_auth_token("34dqxs65C3xapDoCXVWKWMR8hIu_domRdsT4DTvDn3s31iWs")
public_url = ngrok.connect(8000)
print("NGROK PUBLIC URL:", public_url)

def run_api():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run_api).start()


NGROK PUBLIC URL: NgrokTunnel: "https://len-unprecipiced-ai.ngrok-free.dev" -> "http://localhost:8000"


In [ ]:
import requests

url = "https://len-unprecipiced-ai.ngrok-free.dev/analisis"
data = {"ulasan": """UI nya gacor banget"""}

response = requests.post(url, json=data)
print(response.json())



 Teknik Prompt: few_shot_cot
Output mentah:
- "Kualitas Best value phone in the market right now" menyebutkan "Best value", sehingga membahas → Harga
- "Garansi panjang" menyebutkan "Garansi", sehingga membahas → Layanan Penjual
- "Kondisi Barang 100 aman mulus" menyebutkan "Kondisi Barang", sehingga membahas → Desain & Kualitas Bodi
- "hands down sama Poco F7! Performanya mantep abis, multitasking lancar jaya, game berat juga nggak ngelag" menyebutkan "Performanya mantep abis, multitasking lancar jaya, game berat juga nggak ngelag", sehingga membahas → Performa & Gaming
- "Layar AMOLED nya keren banget, warna-warna cerah dan tajam" menyebutkan "Layar AMOLED", sehingga membahas → Layar
- "Kamera 50MP nya juga jos, Foto-foto jernih, detail, dan nggak kalah sama kamera profesional!" menyebutkan "Kamera", "Foto", sehingga membahas → Kamera
- "Baterai 650MAH nya awet banget, gue bisa main game seharian tanpa perlu khawatir kehabisan daya. Dan pengisian cepat 90Watt? Gue bisa balik main da